# Mini	projet	1	:	L’alignement	des	séquences	et	la	programmation	dynamique

***

**PAQUET Michael (000410753) BA-INFO**


# Introduction

Depuis plusieurs années maintenant, et ce dans le domaine biologique, nous sommes témoin d'une explosion des données. 
Devoir manipuler et comparer quelques données, à savoir des protéines ici, s'est transformé en devoir manipuler des dizaines de millier de données. Prenons par exemple une tâche faisant face aux biologistes : ceux-ci doivent aligner des séquences d'acides aminés pour pouvoir déterminer leur ressemblances, que ce doit leur fonction, leur représentation etc.
Avec l'explosion des données, les chercheurs se sont retrouvés avec de plus en plus de séquences et des séquences de plus en plus longues

Il est donc devenu primordial de faire cela de façon numérique en faisant appel à l'informatique. 

Le but de ce projet sera donc d'aligner globalement et localement plusieurs séquences d'acides aminés en approchant les algorithmes de Needleman-Wunsch et de Smith-Waterman, et ce avec des methodes de programmation pertinentes.
Cela nous permettra ainsi d'avoir un alignement optimal global ou local en fonction des besoins de l'utilisateur, et d'ensuite comparer nos résultats avec ceux issus d'algorithmes venant de site tel que LALIGN.

Afin de faire tout cela, nous allons donc procéder comme tel : 
- Création d'ADT aidant à la pertinence et à la structure du code
- Implémentation des algorithmes d'alignement local et global
- Comparaison des résultats obtenus avec ceux d'uniprot et de LALIGN et réponse aux questions posées
- Conclusion sur la globalité du projet



## Implémentation des ADT

Cette première partie portera sur l'explication des choix d'implémentation quand aux ADT exigés dans le cadre du projet.
Ces choix sont aptes à être reconsidérés lors de l'implémentation des prochains projets puisque ceux-ci apporteront de nouvelles problématiques auxquelles nous devrons répondre. 

### ADT Sequence

> Représente une séquence d'acides aminés ainsi que les opérations qu'on peut effectuer sur celle-ci

Cette objet est pour le moment presque vide. Dans le cadre de notre projet, il n'est question que d'aligner les séquences entre elles pour en déterminer plusieurs choses. Je n'ai donc pas trouver d'opérations pertinentes à effectuer sur une telle structure.

Comparé à la structure suivante, à savoir la matrice du substitution, la fonction de parsing n'a ici pas été inclue dans l'ADT.
Ce choix s'explique par le fait qu'un parsing d'un fichier de séquences résulte en plusieurs séquences. Il n'était selon moi pas pertinent qu'un objet "Séquence" aie accès à d'autres séquences. 
Le parsing a donc été fait dans une fonction séparée de l'ADT

Quant au choix de sa représentation, un string a ici été choisi pour sa simplicité et les fonctions pouvant être effectuées. 
Une liste d'une autre instance d'objet "Acide aminé" aurait pu être envisagée. Mais puisqu'on s'apprête à agir sur de très grandes séquences, mon choix s'est orienté vers la solution de complexité minimal, d'autant plus que celle-ci répond à nos besoins.

In [4]:
import numpy as np

class Sequence:

    def __init__(self, sequence):
        self.sequence = sequence

        
    def __repr__(self):
        return "Sequence :\n\n{}\n".format(self.sequence)
    
    def __len__(self):
        return len(self.sequence)
    
    def __getitem__(self, index):
        return self.sequence[index]

        
    def indel(self, position):
        pass
    
    
def parsingSequence(file):
    fichier = open(file, "r")
    reading = False
    sequence = ""
    allSequence = []
    
    for line in fichier:   #Pour chaques lignes dans le fichier
        
        if ">" == line[0] and not reading:  #Croisement du premier >
            reading = True 
            
        elif ">" == line[0] and reading or line[0] == "":   #On était en train de lire et on croise un ">"
            allSequence.append(Sequence(sequence)) 
            sequence = ""
            
        elif reading:     #On continue de lire la séquence
            sequence += line.replace("\n", "")
            
    allSequence.append(Sequence(sequence))
    return allSequence

#### Exemple d'execution

In [5]:
myFile = "WW-sequence.fasta"
sequences = parsingSequence(myFile)
for i in range(len(sequences)):
    print(sequences[i])

Sequence :

VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK

Sequence :

SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP

Sequence :

GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL

Sequence :

EKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSG

Sequence :

SGAKSMWTEHKSPDGRTYYYNTETKQSTWEKPDD

Sequence :

LLSKCPWKEYKSDSGKPYYYNSQTKESRWAKPKE



L'algorithme du parsing ici n'est qu'une simple manipulation de string ligne par ligne. Python rend cela très simple comme vu ci-dessus. Les commentaires à eux seuls permettent de se rendre compte de l'idée derrière l'algorithme.

### ADT Matrice de substitution

> Représente une matrice de substitution ainsi que les opérations qu'on peut effectuer sur celle-ci

Cet objet est initié grâce au parsing d'un fichier texte représentant une matrice de substitution. On peut donc grâce à cet objet représenter une matrice telle que Blosum62, Pam60 etc et effectuer dessus une lecture par exemple.

Une liste de liste, à savoir une matrice, a très logiquement été choisie pour représenter cet objet. Lire une donnée ou trouver les indices d'une valeur s'avère donc très simple.

Comme dit plus haut, contrairement à l'ADT séquence, la methode de parsing est ici inclue dans l'objet puisque celle-ci renvoie une matrice de subsitution choisie par l'utilisateur. 

L'affichage de cet objet est très brut, mais celui-ci n'est pas forcément besoin dans le cadre de ce projet-ci.

In [5]:
class MatriceSub:

    def __init__(self, myFile):
        self.mat = self.parsingMat(myFile)
        self.column = ["A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","B","Z","X","*"]
        self.line = ["A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","B","Z","X","*"]

        
    def __repr__(self):
        return "Matrice de substitution :\n\n{}\n".format(self.mat)


    def parsingMat(self, file):
        file = open(file, "r")
        reading = False
        mat = []
        
        for line in file:  #Pour chaque lignes dans le fichier
            
            if line[0:4] == "   A" or line[0:5] == "    A":   #On détecte le début de la matrice
                reading = True
                
            elif reading:
                
                if line[0] == "*":  #On détecte la fin
                    reading = False
                    
                ligne = line[2:].replace("\n", "").split(" ")
                voidNumber = 0
                
                for i in range(len(ligne)):
                    if ligne[i] == "": 
                        voidNumber += 1
                    else:
                        ligne[i] = int(ligne[i])
                        
                for j in range(voidNumber): #Suppression des caractères vides
                    ligne.remove("")
                    
                mat.append(ligne)
        return mat
    
    def findCoord(self, lineC, columnC):
        x = 0
        y = 0
        for i in range(len(self.line)):
            if self.line[i] == lineC:
                x = i
        for j in range(len(self.column)):
            if self.column[j] == columnC:
                y = j
        return (x, y)

    
    def readMat(self, lineC, columnC):
        coord = self.findCoord(lineC, columnC)
        return self.mat[coord[0]][coord[1]]

#### Exemple d'execution

In [6]:
myFile = "blosum62.txt"
maMatrice = MatriceSub(myFile)
print(maMatrice)
print()
print("Score pour A et C : ", maMatrice.readMat("A", "C"))

Matrice de substitution :

[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1, -1, -2, -1, 1, 0, -3, -2, 0, -2, -1, 0, -4], [-1, 5, 0, -2, -3, 1, 0, -2, 0, -3, -2, 2, -1, -3, -2, -1, -1, -3, -2, -3, -1, 0, -1, -4], [-2, 0, 6, 1, -3, 0, 0, 0, 1, -3, -3, 0, -2, -3, -2, 1, 0, -4, -2, -3, 3, 0, -1, -4], [-2, -2, 1, 6, -3, 0, 2, -1, -1, -3, -4, -1, -3, -3, -1, 0, -1, -4, -3, -3, 4, 1, -1, -4], [0, -3, -3, -3, 9, -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1, -3, -3, -2, -4], [-1, 1, 0, 0, -3, 5, 2, -2, 0, -3, -2, 1, 0, -3, -1, 0, -1, -2, -1, -2, 0, 3, -1, -4], [-1, 0, 0, 2, -4, 2, 5, -2, 0, -3, -3, 1, -2, -3, -1, 0, -1, -3, -2, -2, 1, 4, -1, -4], [0, -2, 0, -1, -3, -2, -2, 6, -2, -4, -4, -2, -3, -3, -2, 0, -2, -2, -3, -3, -1, -2, -1, -4], [-2, 0, 1, -1, -3, 0, 0, -2, 8, -3, -3, -1, -2, -1, -2, -1, -2, -2, 2, -3, 0, 0, -1, -4], [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4, 2, -3, 1, 0, -3, -2, -1, -3, -1, 3, -3, -3, -1, -4], [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2, 4, -2, 2, 0, -3, -2, -1, 

Ici à nouveau l'algorithme de parsing est très similaire à celui des séquences d'acides aminés.

## Implémentation des algorithmes d'alignement

Nous allons maintenant aborder la façon d'implémenter les algorithmes de Needleman-Wunsch et de Smith-Waterman.
Le premier traite de l'alignement global et le second de l'alignement local. 

Afin de faire cela, un nouvel objet appelé Alignement a été créé. Afin de construire celui-ci, plusieurs parramètres seront donnés au constructeur : 
- Les 2 séquences qu'on souhaite aligner
- La matrice de substitution
- Le gap d'entrée
- Le gap d'extension (Non obligatoire dans le cas d'une pénalité linéaire, mais on travaille ici avec une pénalité affine)

Une fois construit on peut appliquer un alignement local ou global sur l'objet. 

Je vais donc maintenant parler des deux algorithmes : 

### Alignement global - Needleman-Wunsch

Puisque nous travaillons avec une pénalité affine qui implique une pénalité d'entrée et d'extension, il faut en plus de la matrice score construire deux matrices supplémentaires car seule la matrice score ne suffit pas à savoir s'il y a eu ou non précédemment un gap :
- La matrice V : Permet de voir s'il y a déjà eu un gap en allant vers le haut 
- La matrice W : Permet de voir s'il y a déjà eu un gap en allant vers la gauche

La construction de nos trois matrices se fait via la fonction initScore et se fait de la façon vue au cours. (plus de commentaires dans le codes sont disponnilbles pour plus d'explications)

Une fois ces trois matrices construites, l'algorithme nous demande uniquement de prendre la valeur tout en bas à droite, et via backtracking de trouver les chemins pour retourner tout en haut à gauche. L'explication de la recherche de ce chemin se trouve dans les commentaires du code.
Le score de l'alignement global de deux séquences sera donc toujours égale à la valeur présente tout en bas à droite de la matrice score. Ces chemins sont trouvés à l'aide des fonctions ***alignGlobal*** et ***findGlobalWays***


### Alignement local - Smith-Waterman

L'alignement fonctionne presque comme l'alignement global à la différence que :
- Les trois matrices sont initées à 0 partout
- Il n'y a jamais de valeurs négatives dans les matrices
- Le début de la recherche du chemin commence au maximum de la matrice score
- Il n'est pas partinent de trouver plusieur chemin pour un même score, il n'y aura donc pas notion de backtracking
- Lorsqu'on trouve un chemin, les valeurs trouvées sont mises à 0 et un recalcul de la matrice est effectué
- Un nouveau maximum est alors trouvé pour trouver un nouvel alignement local

Des fonctions différentes de l'alignement global ont donc été implémentées, il s'agit d' ***alignLocal*** et ***findLocalWays***

> NB : Désolé d'avance pour le gros bloc de code qui suit. Il s'agit là d'un ADT et je ne peux donc pas le séparer en plusieurs blocs de code. C'est pourquoi j'ai choisi d'expliquer les deux algorithmes d'une traite avant de présenter ce dit-bloc. 

In [7]:
class Alignement:

    def __init__(self, seq1, seq2, t, gap, extensionGap = 0):
        self.seq1 = seq1
        self.seq2 = seq2
        self.t = t
        self.gap = gap
        self.E = extensionGap
        self.V = []
        self.W = []
        self.score = []
        self.chemin = []
        self.globalCount = 0


    def alignGlobal(self, k):
        self.V, self.W, self.score = self.initScore(False)
        solution = [self.score[-1][-1]]
        align1 = "" #Align1 et align2 sont les string représentant les séquences avec les gaps présents "-"
        align2 = ""
        lien = ""
        print("Score de l'alignement : ", solution[0], "\n")
        self.findGlobalWays(len(self.seq1), len(self.seq2), align1, align2, lien, solution, k)
        self.globalCount = 0


    def alignLocal(self, k):
        self.V, self.W, self.score = self.initScore(True)
        for i in range(k):
            solution = []
            solution.append(np.max(self.score)) #Recherche du max
            print("Score de l'alignement :", solution[0], "\n")
            i, j = np.where(self.score==solution[0]) #On prend l'indice de ce max
            i = i[0] #Il est possible qu'il y ai un max à plusieurs endroits
            j = j[0] #on choisi ici de prendre le premier
            #print(i, j)
            align1 = ""
            align2 = ""
            lien = ""
            self.findLocalWays(int(i), int(j), align1, align2, lien, solution)


    def initScore(self, isLocal):
        score = []
        V = []
        W = []
        sentinelle = -99999
        

        for i in range(len(self.seq1)+1):
            score.append([0 for j in range(len(self.seq2)+1)])
            V.append([0 for j in range(len(self.seq2)+1)])
            W.append([0 for j in range(len(self.seq2)+1)])
            #Matrices toutes initées à 0 partout
            if not isLocal:
                score[i][0] = self.gap - (i-1)


        if not isLocal: #S'il s'agit d'un alignement global, il faut ajouter les gaps et valeurs sentinelles 
            score[0] = [self.gap - (i-1) for i in range(len(self.seq2)+1)]
            score[0][0] = 0
            
            #Construction de V
            for i in range(1, len(self.seq1)+1):
                V[i][0] = self.gap - (i-1)
            V[0] = [sentinelle for i in range(len(self.seq2)+1)]
            V[0][0] = 0
            
            #Construction de W
            for i in range(1, len(self.seq1)+1):
                W[i][0] = sentinelle
            W[0] = [self.gap - (i-1) for i in range(len(self.seq2)+1)]
            W[0][0] = 0
        

        s1 = 0
        for i in range(1, len(self.seq1)+1):
            s2 = 0
            for j in range(1, len(self.seq2)+1):
                #Calcul de l'intérieur des matrices selon l'algorithme
                if isLocal:
                    #Pas de valeur négative si local -> ajout de 0 en paramètre
                    V[i][j] = max(score[i-1][j] + self.gap, V[i-1][j] - self.E, 0)
                    W[i][j] = max(score[i][j-1] + self.gap, W[i][j-1] - self.E, 0)
                    score[i][j] = max(V[i][j], W[i][j], score[i-1][j-1] + self.t.readMat(self.seq1[s1], self.seq2[s2]), 0)
                    
                else:
                    V[i][j] = max(score[i-1][j] + self.gap, V[i-1][j] - self.E)
                    W[i][j] = max(score[i][j-1] + self.gap, W[i][j-1] - self.E)
                    score[i][j] = max(V[i][j], W[i][j], score[i-1][j-1] + self.t.readMat(self.seq1[s1], self.seq2[s2]))
                    
                s2 += 1
            s1 += 1
        return (V, W, score)


    def findGlobalWays(self, i, j, align1, align2, lien, solution, k):
        
        if self.globalCount == k:
            return True
        
        if i == 0 and j == 0:  #Condition d'arrêt
            #print(solution)
            print(align1[::-1])
            print(lien[::-1])
            print(align2[::-1])
            self.globalCount += 1
        
        else:
            if i == 0:   #Si on se trouve sur un bord
                solution.append(self.score[i][j-1])
                align1 += self.seq2[j-1]
                align2 += "-"
                lien += " "
                self.findGlobalWays(i, j-1, align1, align2, lien, solution, k)
                solution.remove(solution[-1])  #Déconstruction
                
            elif j == 0:   #L'autre bord
                solution.append(self.score[i-1][j])
                align1 += "-"
                align2 += self.seq1[i-1]
                lien += " "
                self.findGlobalWays(i-1, j, align1, align2, lien, solution, k)
                solution.remove(solution[-1])
                
            else: 
                
                
                if self.score[i-1][j-1] + self.t.readMat(self.seq1[i-1], self.seq2[j-1]) == self.score[i][j]:#On prend la diagonale
                    solution.append(self.score[i-1][j-1])
                    align1 += self.seq2[j-1]
                    align2 += self.seq1[i-1]
                    if self.seq1[i-1] == self.seq2[j-1]:
                        lien += ":"
                    elif self.t.readMat(self.seq1[i-1], self.seq2[j-1]) >= 0:
                        lien += "."
                    else:
                        lien += " "
                    self.findGlobalWays(i-1, j-1, align1, align2, lien, solution, k)
                    solution.remove(solution[-1])
                
                if self.V[i][j] == self.score[i][j]:   #On va vers le haut
                    solution.append(self.score[i-1][j])
                    align1 += "-"
                    align2 += self.seq1[i-1]
                    lien += " "
                    self.findGlobalWays(i-1, j, align1, align2, lien, solution, k)  
                    solution.remove(solution[-1])            
                
                
                if self.W[i][j] == self.score[i][j]:  #On va vers la gauche
                    solution.append(self.score[i][j-1])
                    align1 += self.seq2[j-1]
                    align2 += "-"
                    lien += " "
                    self.findGlobalWays(i, j-1, align1, align2, lien, solution, k)
                    solution.remove(solution[-1]) 
                
                
            

    def findLocalWays(self, i, j, align1, align2, lien, solution):
        #Fonctionne comme pour le global à la différence qu'il n'y a plus de backtracking, et que les valeurs du chemin trouvé
        #doivent être mises à 0. Une fois la solution trouvé, on recalcul la "sous matrice"
        self.chemin.append((i, j))
        if self.score[i][j] <= 0:
            print("CHEMIN TROUVÉ : \n")
            #print(solution)
            print(align1[::-1])
            print(lien[::-1])
            print(align2[::-1])
            #printMat(self.score)
            self.reCalculate(self.chemin[-2][0], self.chemin[-2][1])  #On recalcule la sous matrice à partir du point précédent
            print("\n")                                               #trouvé dans le chemin
            #printMat(self.score)
        
        else:
            #Plus de if if if mais if elif elif car pas de backtracking
            if i == 0:
                solution.append(self.score[i][j-1])
                align1 += self.seq2[j-1]
                align2 += "-"
                lien += " "
                self.score[i][j] = 0
                self.V[i][j] = 0
                self.W[i][j] = 0
                self.findLocalWays(i, j-1, align1, align2, lien, solution)
                
            elif j == 0:
                solution.append(self.score[i-1][j])
                align1 += "-"
                align2 += self.seq1[i-1]
                lien += " "
                self.score[i][j] = 0
                self.V[i][j] = 0
                self.W[i][j] = 0
                self.findLocalWays(i-1, j, align1, align2, lien, solution)
                
            else:
                    
                if self.score[i-1][j-1] + self.t.readMat(self.seq1[i-1], self.seq2[j-1]) == self.score[i][j]:
                    solution.append(self.score[i-1][j-1])
                    align1 += self.seq2[j-1]
                    align2 += self.seq1[i-1]
                    if self.seq1[i-1] == self.seq2[j-1]:
                        lien += ":"
                    else:
                        lien += "."
                    self.score[i][j] = 0
                    self.V[i][j] = 0
                    self.W[i][j] = 0
                    self.findLocalWays(i-1, j-1, align1, align2, lien, solution)

                elif self.W[i][j] == self.score[i][j]:
                    solution.append(self.score[i][j-1])
                    align1 += self.seq2[j-1]
                    align2 += "-"
                    lien += " "
                    self.score[i][j] = 0
                    self.V[i][j] = 0
                    self.W[i][j] = 0
                    self.findLocalWays(i, j-1, align1, align2, lien, solution)

                elif self.V[i][j] == self.score[i][j]:
                    solution.append(self.score[i-1][j])
                    align1 += "-"
                    lien += " "
                    align2 += self.seq1[i-1]
                    self.score[i][j] = 0
                    self.V[i][j] = 0
                    self.W[i][j] = 0
                    self.findLocalWays(i-1, j, align1, align2, lien, solution)  


    def reCalculate(self, k, m):
        #fonction de recalcul
        for i in range(k, len(self.seq1)+1):
            for j in range(m, len(self.seq2)+1):
                if (i, j) not in self.chemin: #On ne recalcul pas les valeurs si celles-ci étaient présentes dans le chemin trouvé
                    self.V[i][j] = max(self.score[i-1][j] + self.gap, self.V[i-1][j] - self.E, 0)
                    self.W[i][j] = max(self.score[i][j-1] + self.gap, self.W[i][j-1] - self.E, 0)
                    self.score[i][j] = max(self.V[i][j], self.W[i][j], self.score[i-1][j-1] + self.t.readMat(self.seq1[i-1], self.seq2[j-1]), 0)

A noté ici que la fonction de recalcul gaspille du temps d'execution. En effet, le code peut s'arrêter une fois qu'il détecte que les valeurs dans S, V et W n'ont pas été changées. Ceci n'a pas été implémenté ici par soucis de faciliter, mais aurait pu/dû être fait.

#### Exemples d'execution

> ":" = match trouvé, "." = mutation acceptée

In [8]:
seq1 = "ISALIGNED" 
seq2 = "THISLINE"
k = 2
align = Alignement(seq1, seq2, maMatrice, -4, 1)
align.alignGlobal(k)

Score de l'alignement :  10 

THIS-LI-NE-
  :: :: :: 
--ISALIGNED


In [9]:
align.alignLocal(k)

Score de l'alignement : 19 

CHEMIN TROUVÉ : 

IS-LI-NE
:: :: ::
ISALIGNE


Score de l'alignement : 5 

CHEMIN TROUVÉ : 

IN
:.
IS





## Comparaison des résultats, réponse aux questions

Nous allons ici répondre aux demandes du projet, à savoir la résolution de questions et la comparaison/observation de nos résultats.

### Alignement global

Comparons d'abord nos résultats avec ceux de LALIGN vis à vis des séquences prises dans WW-séquence.fasta.
Je choisis ici de travailler avec la matrice de substitution Blosum62, une pénalité d'entrée I = -4 et une pénalité d'extension E = -1

>Blosum indique la similarité entre 2 séquences. Une matrice blosum62 a été construite sur base de séquences présentant 62% de similarité. En utilisant cette matrice, on veut donc que nos séquences présentent le même degré de similarité. 
Pour PAM, il s'agit du nombre de résidu muté sur 100. PAM60 indique donc qu'elle a été construite sur des séquences ayant 60 résidus mutés sur 100 durant l'évolution. 



In [10]:
myFile = "WW-sequence.fasta"
sequences = parsingSequence(myFile)

myFile = "blosum62.txt"
maMatrice = MatriceSub(myFile)

for i in range(len(sequences)):
    seq1 = sequences[i]
    for j in range(i+1, len(sequences)):
        seq2 = sequences[j]
        print("Alignement de la séquence numéro ", i+1, " et ", j+1)
        align = Alignement(seq1, seq2, maMatrice, -4, 1)
        align.alignGlobal(1)
        print("\n\n")

Alignement de la séquence numéro  1  et  2
Score de l'alignement :  77 

SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP
 ::: :::  .   :. :..:: .. : :. :  
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK



Alignement de la séquence numéro  1  et  3
Score de l'alignement :  90 

GPLPPGWE-ERTHTDGRIFYINHNI-KRTQWEDPRL
 ::: :::  .: ..:. ...:: : . : :.::: 
VPLPAGWEMAKT-SSGQRYFLNH-IDQTTTWQDPRK



Alignement de la séquence numéro  1  et  4
Score de l'alignement :  82 

EKLPPGWEKRMSR-SSGRVYYFNHITNASQWERPSG
  :: :::  :.. :::. :..::: ... :. :  
VPLPAGWE--MAKTSSGQRYFLNHIDQTTTWQDPRK



Alignement de la séquence numéro  1  et  5
Score de l'alignement :  43 

----SGAKSMWTEHKSPDGRTYYYN--TETKQSTWEKPDD
    .:    :   :. .:. :. :   .:  .::. :  
VPLPAG----WEMAKTSSGQRYFLNHIDQT--TTWQDPRK



Alignement de la séquence numéro  1  et  6
Score de l'alignement :  54 

L-LSKCP--WKEYKSDSGKPYYYN--SQTKESRWAKPKE
. :   :  :.  :..::. :. :  .::  . :  :..
VPL---PAGWEMAKTSSGQRYFLNHIDQT--TTWQDPRK



Alignement de la séquence numéro  2  et  3
Score d

Comparons maintenant quelques-uns de ces résultats avec LALIGN pour ces mêmes paramètres.

***Séquence 1 et 2***

```
n-w opt:  77  Z-score: 123.8  bits: 24.7 E(1): 7.9e-14
global/global (N-W) score: 77; 38.2% identity (58.8% similar) in 34 aa overlap (1-34:1-34)

               10        20        30    
unknow VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
        ::: :::  .   :. :..:: .. : :. :  
unknow SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP
               10        20        30   
```

***séquence 2 et 3***

```
n-w opt: 115  Z-score: 161.5  bits: 31.6 E(1): 3.4e-29
global/global (N-W) score: 115; 55.6% identity (72.2% similar) in 36 aa overlap (1-34:1-34)

               10          20        30    
unknow SPLPPGWEERQ--DILGRTYYVNHESRRTQWKRPTP
       .:::::::::   :  :: .:.::. .::::. :  
unknow GPLPPGWEERTHTD--GRIFYINHNIKRTQWEDPRL
               10          20        30    
```

***séquence 4 et 5***

```
n-w opt:  60  Z-score: 115.9  bits: 23.2 E(1): 2.2e-11
global/global (N-W) score: 60; 34.3% identity (60.0% similar) in 35 aa overlap (1-35:1-34)

               10        20        30     
unknow EKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSG
       .     : .. :  .::.::.:  :. : ::.:. 
unknow SGAKSMWTEHKS-PDGRTYYYNTETKQSTWEKPDD
               10         20        30 
```

Pour la séquence 1 et 2, on se rend compte que le résultat est exactement le même.
Mais cela ne permet pas de faire une généralisation quand à la ressemblance des résultats.

En effet, si nous regardons maintenant les séquences 2 et 3, nous avons de petites divergences : 
- Le score est légèrement différent
- Le chemin n'est pas tout à fait le même (Divergence après le 10ème acide aminé)

Pour ce qui est du score, tout ce qu'on peut faire est émettre des hypothèses. Peut-être utilisent-ils un algorithme légèrement différent du notre pour calculer la matrice, ou alors leur matrice Blosum62 est-elle exactement pareil à la notre ? (peu probable)
Ne disposant pas de ce genre d'informations, il serait difficile d'en déduire une conclusion.

Quant au chemin maintenant, une petite différence comme on la voit pourrait s'expliquer de la façon suivante : 
Dans notre fonction ***findGlobalWays***, nous cherchons d'abord un match avec la diagonale, puis V, puis W. 
Si je décide de changer cet ordre, le résultat sera différent. (Methode testée)
On peut donc émettre l'idée que LALIGN prend un ordre différent du mien. S'en suit la question :

Un ordre est-il plus pertinent qu'un autre ? 

D'après moi, L'ordre entre V et W importe peu, mais prendre la diagonale semble être le meilleur choix si celui-ci est disponnible pour éviter un maximum de gap.

L'alignement des séquences 4 et 5 montre quant à lui une différence de score mais pas de chemin.

Biensur, cela n'est pas à 100% fiable. Il se peut également, tout comme pour la différence de score, que la différence ne réside pas là mais en une différence de methode utilisée, ou encore une matrice blosum62 différente ayant un impact sur le chemin trouvé. 

On pourrait évidemment comparer beaucoup plus de séquences pour essayer de situer et expliquer plus en détails les différences.


Essayons maintenant de répondre à la question qui nous a été posée : ***Quelles séquences sont les plus similaires ?***

Pour cela, on peut regarder nos résultats un peu plus haut. Nous observons que le plus haut score d'alignement revient à l'alignement des séquences 2 et 3. Qui plus est, LALIGN indique le même résultat à peu de choses près.
Essayons de donner plus de valeur à ces résultats en les testant avec une autre matrice de substitution, blosum80 par exemple :

In [11]:
myFile = "WW-sequence.fasta"
sequences = parsingSequence(myFile)

myFile = "blosum80.txt"
maMatrice = MatriceSub(myFile)

for i in range(len(sequences)):
    seq1 = sequences[i]
    for j in range(i+1, len(sequences)):
        seq2 = sequences[j]
        print("Alignement de la séquence numéro ", i+1, " et ", j+1)
        align = Alignement(seq1, seq2, maMatrice, -4, 1)
        align.alignGlobal(1)
        print("\n\n")

Alignement de la séquence numéro  1  et  2
Score de l'alignement :  76 

SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP
 ::: :::  .   :. :..::  . : :. :  
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK



Alignement de la séquence numéro  1  et  3
Score de l'alignement :  92 

GPLPPGWE-ERTHTDGRIFYINHNI-KRTQWEDPRL
 ::: :::  .: . :. ...:: : . : :.::: 
VPLPAGWEMAKT-SSGQRYFLNH-IDQTTTWQDPRK



Alignement de la séquence numéro  1  et  4
Score de l'alignement :  88 

EKLPPGWEKRMSR-SSGRVYYFNHITNASQWERPSG
  :: :::  :.. :::. :..::: ... :. :  
VPLPAGWE--MAKTSSGQRYFLNHIDQTTTWQDPRK



Alignement de la séquence numéro  1  et  5
Score de l'alignement :  43 

----SGAKSMWTE-HKSPDGRTYYYNTETKQ-STWEKPDD
    .:    : :  :.  :. :. : .  : .::. :  
VPLPAG----W-EMAKTSSGQRYFLN-HIDQTTTWQDPRK



Alignement de la séquence numéro  1  et  6
Score de l'alignement :  55 

L-LSKCP--WKEYKSDSGKPYYYN--SQTKESRWAKPKE
. :   :  :.  :. ::. :. :   ::  . :  :..
VPL---PAGWEMAKTSSGQRYFLNHIDQT--TTWQDPRK



Alignement de la séquence numéro  2  et  3
Score d

On retrouve en effet la même conclusion. Un test avec pam60 et pam120 a bien évidemment aussi été effectué (il suffit pour cela de remplacer le nom du fichier) et le résultat est le même.

On peut donc en conclure que les séquences les plus similaires sont les ***séquences 2 et 3*** à savoir :
- SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP
- GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL

Mais qu'en est-il de ces deux séquences ? Maintenant qu'on a jugé ces deux séquences similaires, il serait pertinent de voir si elles viennent de la même protéine ou non. Regardons cela avec Uniprot.

Lorsqu'on rentre l'ID de la séquence dans Uniprot, on voit bien que ces séquences appartiennent bien à la même protéine à savoir la ***E3 ubiquitin-protein ligase NEDD4***.

Ces 2 séquences sont en fait deux domaines (1) ***WW*** de la même protéine. Les domaines WW sont des domaines dans lesquels deux acides aminés "W" sont présents à 20-22 de distance l'un de l'autre. Celui-ci se plie généralement en "meandering triple-stranded beta sheet" (2)

(1) Un domaine est une séquence d'acides aminés qui peut exister indépendemment de la protéine dans laquelle est est. Chaque domaine forme une structure tri-dimensionnel (Depuis https://en.wikipedia.org/wiki/Protein_domain)

(2) Information tirée de Wikipedia : https://en.wikipedia.org/wiki/WW_domain



### Alignement local

De la même façon que l'alignement global, procédons par étape et commençons par comparer nos résultats avec ceux de LALGIN.


In [12]:
myFile = "protein-sequences.fasta"
sequences = parsingSequence(myFile)

myFile = "blosum62.txt"
maMatrice = MatriceSub(myFile)

seq1 = sequences[0]
seq2 = sequences[3]

align = Alignement(seq1, seq2, maMatrice, -4, 1)
align.alignLocal(3)

Score de l'alignement : 149 

CHEMIN TROUVÉ : 

DEEKLPPGWEKRMSR-SSGRVYYFNHITNASQW-E--R--------------PS--G--NSSSGG-KNGQGEPARVRCSHL-LVKHSQSRRPSSW-------R---QEKITRT---KE------EALE--LI---NG------YIQKIKSGEED--F---ESL--ASQ-F--SDCSSAK----A-RGDLGAFSR-GQMQKPFEDASFA--LRTGEM--SGPVFTDSG-IHIILRTE
:...::.:::  :.. :::..:..:::.....: .  .              :.  .  ::.::. ..: .:.:....... ...:  .....::       :   ...:...   :.      ....  ..   :.      ..:......:.  .   :.:  :.. .  :...:.:    : :..:..... :..:.:.......  :::...  :.: :..:: .:  .:.:
DDVPLPAGWE--MAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNMMNSASGPLPDG-WEQAMTQDGEIYYINH--KNKTTSWLDPRLDPRFAMNQRISQSAPVKQPPPLAPQSPQGGVMGGSNSNQQQQMRLQQLQMEKERLRLKQQELLRQAMRNINPSTANSPKCQELALRSQLPTLEQDGGTQNPVSSPGMSQELRTMTTNSSDP-FLNSGTYH--SRDE


Score de l'alignement : 141 

CHEMIN TROUVÉ : 

LPPGWEKRMSRSSGRVYYFNHITNASQW--ER--P--------SGNS--------S--S------GGKN-G----------QGEPARVRCSHL-LVKHSQSRR--PS---SWR-QEKITRTKEEALELINGYIQK-IKS-G-EEDFESLASQFSD---CSSA-KAR--G-DLG-AFSRGQMQK-PFEDASF

#### Comparaison avec LALIGN

```
>>unknown 163 bp                                          (163 aa)
 Waterman-Eggert score: 125;  24.9 bits; E(1) <  0.0027
24.0% identity (51.6% similar) in 192 aa overlap (232-396:7-163)

             240        250       260       270       280       290
unknow LPDGWEQAMTQD-GEIYYINHKNKTTSWLDPRLDPRFAMNQRISQSAPVKQPPPLAPQSP
       :: :::. :... :..::.:: .....:            .:                 :
unknow LPPGWEKRMSRSSGRVYYFNHITNASQW------------ER-----------------P
         10        20        30                                    

               300        310                   320         330    
unknow QGGVM-GGSNSN-QQQQMRLQQL-----QMEK------ERL-RLKQQ--ELLRQAMRNIN
       .:.   ::.:.. .  ..: ..:     : ..      :.. : :..  ::..  ...:.
unknow SGNSSSGGKNGQGEPARVRCSHLLVKHSQSRRPSSWRQEKITRTKEEALELINGYIQKIK
        40        50        60        70        80        90       

                340        350       360       370       380       
unknow P------STANS-PKCQELALRSQLPTLEQDGGTQNPVSSPGMSQELRTMTTNSSDP-FL
              : :..   :..   :..: .... :  :.: .. ...  ::  : . : : : 
unknow SGEEDFESLASQFSDCSSAKARGDLGAFSR-GQMQKPFEDASFA--LR--TGEMSGPVFT
       100       110       120        130       140           150  

        390        
unknow NSGTYHS--RDE
       .::  :   : :
unknow DSGI-HIILRTE
             160   

>--
 Waterman-Eggert score: 114;  22.8 bits; E(1) <  0.011
23.2% identity (49.1% similar) in 228 aa overlap (167-376:1-162)

        170         180       190       200       210       220    
unknow IPDDVPLPAGWE--MAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNM
       . :.  :: :::  :.. :::. :..::: ... :.  :           :..       
unknow MADEEKLPPGWEKRMSR-SSGRVYYFNHITNASQWE--R-----------PSG-------
               10         20        30                             

          230        240       250         260       270       280 
unknow MNSASGPLPDGW-EQAMTQDGEIYYINHKN--KTTSWLDPRLDPRFAMNQRISQSAPVKQ
        ::.::   .:  : : .. ...  ..:..  . .::   :       ...:...   :.
unknow -NSSSGG-KNGQGEPARVRCSHLL-VKHSQSRRPSSW---R-------QEKITRT---KE
       40         50        60         70                  80      

             290       300       310       320       330       340 
unknow PPPLAPQSPQGGVMGGSNSNQQQQMRLQQLQMEKERLRLKQQELLRQAMRNINPSTANSP
           : .     ...:          .:...  .: .    ..:  :       : ..: 
unknow E---ALE-----LINGY---------IQKIKSGEEDF----ESLASQF------SDCSSA
                    90                100           110            

                350                360        370      
unknow KCQ-EL-AL-RSQL--PTLEQD-------GGTQNPV-SSPGMSQELRT
       :.. .: :. :.:.  : .: :       :  ..:: .. :.   :::
unknow KARGDLGAFSRGQMQKP-FE-DASFALRTGEMSGPVFTDSGIHIILRT
        120       130         140       150       160  

>--
 Waterman-Eggert score: 73;  15.3 bits; E(1) <  0.87
26.3% identity (45.5% similar) in 167 aa overlap (21-161:32-161)

                 30         40        50            60             
unknow SQ--PPQGQGPPSGP-GQPAPAATQAAPQAPPAGHQIV-HV---RGDS---E----TDLE
       ::   :.:..  .:  ::  :: ....       : .: :    : .:   :    :  :
unknow SQWERPSGNSSSGGKNGQGEPARVRCS-------HLLVKHSQSRRPSSWRQEKITRTKEE
              40        50               60        70        80    

         70        80        90       100       110       120      
unknow ALFNAVMNPKTANVPQTVPMRLRKLPDSFFKPPEPKSHSRQASTDAGTAGALTPQHVRAH
       ::   ..:   . . :    ....  ..:      .: . : : :...: :      :. 
unknow AL--ELIN---GYI-Q----KIKSGEEDF------ESLASQFS-DCSSAKA------RG-
             90               100             110              120 

        130       140                 150         160 
unknow SSPASLQLGAVSPGTLT-P---------TGVVSGPAATPTAQH--LR
             .::: : : .  :         :: .:::. : .. :  ::
unknow ------DLGAFSRGQMQKPFEDASFALRTGEMSGPVFTDSGIHIILR
                    130       140       150       160 
```


On remarque énormément de différences entre les deux résultats. Si une similarité est à notée, c'est que notre deuxième alignement ressemble légèrement au premier alignement trouvé par LALIGN. Les scores tout comme les chemins divergent beaucoup.
De plus, mais cela est anecdotique, on remarque que la façon d'afficher le résultat est beaucoup mieux travaillé chez LALIGN que dans mon projet.
Comment expliquer ces différences ?

Observations :

tout d'abord, toutes les hypothèses posées pour expliquer les différences existentes plus haut dans l'alignement global sont toujours de mises ici ! 

Enfin, et à ne pas sous-estimer, il peut s'agir d'une erreur de programmation de ma part qui influe directement les résultats.

> D'autres comparaisons avec d'autres protéines ont aussi été effectuées mais elles amenaient aux même genre d'observations, c'est pourquoi je ne présente ici que ce résultat.

En alignant localement nos séquences, nous trouvons des similarités entre elles. Essayons de comprendre ces similarités grâce aux informations fournies par UniProt.

Nous avons donc 4 protéines : 
- Pre-mRNA-processing factor 40 homolog A
- Transcriptional coactivator YAP1
- E3 ubiquitin-protein ligase NEDD4 (vue précédemment)
- Peptidyl-prolyl cis-trans isomerase NIMA-interacting 1

Étant novice avec Uniprot, je vais essayer d'évoquer le plus d'informations pouvant expliquer les similarités trouvées avec notre algorithme local.

##### Fonction moléculaire

On remarque que la Pre-mRNA-processing factor 40 homolog A et la E3 ubiquitin-protein ligase NEDD4 partagent la fonction ***proline-rich region binding***

La E3 ubiquitin-protein ligase NEDD4 et la Pre-mRNA-processing factor 40 homolog A partagent la fonction ***proline-rich region binding***

##### Processus biologique

La Pre-mRNA-processing factor 40 homolog A et la Peptidyl-prolyl cis-trans isomerase NIMA-interacting 1 partagent le processus de ***regulation of cytokinesis***

##### Domaine et famille

On observe que toutes les protéines comportent des domaines WW ! Leur nombre varie de 1 à 4. 

#### Structure secondaire :

***Transcriptional coactivator YAP1***

<img src="StructYAP1.png",width=1000,height=300>

***E3 ubiquitin-protein ligase NEDD4***

<img src="StructNEDD4.png",width=1000,height=100>

***Pre-mRNA-processing factor 40 homolog A***

<img src="StructPRPF.png",width=1000,height=100>

***Peptidyl-prolyl cis-trans isomerase NIMA-interacting 1***

<img src="StructPIN1.png",width=1000,height=100>



Il est aisé de remarquer que toutes nos protéines présentent des hélices, des batonnets et des "Turn". 


### Conclusion sur les observations

Avec toutes les informations pêchées sur UniProt, on peut expliquer les similaritées trouvées dans nos protéines grâce à l'alignement local. A noté ici qu'il est en fait difficile de voir les similarités dans nos alignement puisque ceux-ci sont fort longs. Il serait peut-être judicieux d'effectuer un alignement local avec un paramètre k relativement haut. Ainsi nos alignements serait de plus en plus petits et nous pourrions situer plus précisément quelques similarités.

Voici par exemple un alignement trouvé après plus de 30 itérations sur l'alignement local entre PIN1_HUMAN et YAP1_HUMAN :

```
PSGNSS--SGGKNGQGEPA
:.:...  :....:::.:.
PQGQGQPPSQPPQGQGPPS
```
On pourrait alors chercher les propriétés de ce genre de séquences et ainsi expliquer la similarité. 

>Le code si dessous execute l'alignement avec un grand paramètre k dont on parle juste en haut. J'ai juste mis la ligne d'execution principale en commentaire car l'execution tourne pendant une bonne minute avant de faire les 40 alignements. Mais il est tout à fait possible de l'executer.



In [13]:
myFile = "protein-sequences.fasta"
sequences = parsingSequence(myFile)

myFile = "blosum62.txt"
maMatrice = MatriceSub(myFile)

seq1 = sequences[0]
seq2 = sequences[3]

align = Alignement(seq1, seq2, maMatrice, -4, 1)
#align.alignLocal(40)

# Conclusion

Pour terminer ce projet, on peut donc conclure que la problématique est bel et bien résolue. Nous avons pu à l'aide de traitement numérique déterminer les similarités entre plusieurs séquences en les alignant, ce qu'on a prouvé en observant nos résultats et en les comparant avec le site de LALIGN et d'UniProt.
Attention tout de même à notre alignement local qui diverge beaucoup de l'alignement local provenant de LALIGN. 